In [9]:
import pandas as pd
import matplotlib.pyplot as plt

In [42]:
# Import data
df_other = pd.read_csv('Other-Implementations/logs/performance_log.csv')
df_metal = pd.read_csv('metal-logs/metal_performance_log.csv')

# Combine both datasets into one DataFrame
df_other['Source'] = 'Other'
df_metal['Source'] = 'Metal'
df_combined = pd.concat([df_other, df_metal], ignore_index=True)

# Group by ExecutionType, NumIterations, and Source, and calculate mean for both columns
grouped = df_combined.groupby(['Source', 'ExecutionType', 'NumIterations'])[['ReconstructionTime_ms', 'FinalErrorNorm']].mean().reset_index()

# Pivot the data to create a single DataFrame
pivoted = grouped.pivot(index='NumIterations', columns=['Source', 'ExecutionType'], values=['ReconstructionTime_ms', 'FinalErrorNorm'])

# Flatten the multi-level columns for easier readability
pivoted.columns = ['_'.join(col).strip() for col in pivoted.columns.values]
pivoted = pivoted.rename_axis('Iterations', axis='index')

pivoted = pivoted.rename(columns={
    'ReconstructionTime_ms_Other_Sequential': 'Seq. (ms)',
    'ReconstructionTime_ms_Other_openmp': 'OMP (ms)',
    'FinalErrorNorm_Other_Sequential': 'Seq. Error Norm',
    'FinalErrorNorm_Other_openmp': 'OMP Error Norm',
    'ReconstructionTime_ms_Metal_Metal': 'Metal (ms)',
    'FinalErrorNorm_Metal_Metal': 'Metal Error Norm'
})

# Define the new column order
new_column_order = [
    'Seq. (ms)',         
    'OMP (ms)',    
    'Metal (ms)',    
    'Seq. Error Norm',     
    'OMP Error Norm',        
    'Metal Error Norm'   
]

# Reorder the columns
pivoted = pivoted[new_column_order]

# Add speedup columns
pivoted['Speedup: OMP vs Seq'] = pivoted['Seq. (ms)'] / pivoted['OMP (ms)']
pivoted['Speedup: Metal vs Seq'] = pivoted['Seq. (ms)'] / pivoted['Metal (ms)']
pivoted['Speedup: Metal vs OMP'] = pivoted['OMP (ms)'] / pivoted['Metal (ms)']

# Calculate the global minimum and maximum across the selected columns
vmin = pivoted[['Speedup: OMP vs Seq', 'Speedup: Metal vs Seq', 'Speedup: Metal vs OMP']].min().min() 
vmax = pivoted[['Speedup: OMP vs Seq', 'Speedup: Metal vs Seq', 'Speedup: Metal vs OMP']].max().max()

styled_table = pivoted.style.format(precision=3)\
    .set_table_styles([
        {'selector': 'thead th', 'props': [('font-family', 'Georgia'), ('font-size', '12px'), ('font-weight', 'bold')]},
        {'selector': 'tbody td', 'props': [('font-family', 'Georgia'), ('font-size', '12px')]},
        {'selector': 'table', 'props': [('border-collapse', 'collapse'), ('width', '100%')]}
    ])
styled_table = styled_table.set_table_styles([
    {'selector': 'thead th', 'props': [('border', '1px solid black'), ('padding', '8px')]},
    {'selector': 'tbody td', 'props': [('border', '1px solid black'), ('padding', '8px')]},
    {'selector': 'table', 'props': [('border', '2px solid black')]}
])

# Apply the gradient with consistent vmin and vmax
styled_table = pivoted.style.format(precision=3)\
    .background_gradient(subset=['Speedup: OMP vs Seq', 'Speedup: Metal vs Seq', 'Speedup: Metal vs OMP'], cmap='Blues', vmin=vmin, vmax=vmax)
styled_table
# Drop the norm columns for plotting
# pivoted_plot = pivoted.drop(columns=['Seq. Error Norm', 'OMP Error Norm','Metal Error Norm'])
# pivoted_plot = pivoted_plot.reset_index()

# styled_table.to_html('table3.html')


,Seq. (ms),OMP (ms),Metal (ms),Seq. Error Norm,OMP Error Norm,Metal Error Norm,Speedup: OMP vs Seq,Speedup: Metal vs Seq,Speedup: Metal vs OMP
Iterations,,,,,,,,,
1,126.711,71.500,14.016,0.996,0.996,0.996,1.772,9.040,5.101
10,1231.700,732.940,67.532,0.965,0.965,0.965,1.680,18.239,10.853
100,20751.050,7805.349,546.398,0.808,0.808,0.808,2.659,37.978,14.285
500,129114.500,39806.083,2400.600,0.661,0.661,0.661,3.244,53.784,16.582
1000,226751.500,88153.525,4874.492,0.576,0.576,0.576,2.572,46.518,18.085
5000,nan,nan,23092.400,nan,nan,0.338,nan,nan,nan
10000,nan,nan,47546.700,nan,nan,0.254,nan,nan,nan
15000,nan,nan,74106.900,nan,nan,0.222,nan,nan,nan
20000,nan,nan,97672.700,nan,nan,0.208,nan,nan,nan


In [ ]:
# Import data
df_other = pd.read_csv('Other-Implementations/logs/normalisation_log.csv')
df_metal = pd.read_csv('metal-logs/normalisation_log.csv')

# Combine both datasets into one DataFrame
df_metal['Source'] = 'Metal'
df_other['Source'] = 'Other'
df_combined = pd.concat([df_metal, df_other], ignore_index=True)

# Group by Type, ImageWidth,NumAngles,NumDetectors
grouped = df_combined.groupby(['Type', 'ImageWidth', 'NumAngles', 'NumDetectors'])[['NormalisationTime']].mean().reset_index()

# Pivot the data to create a single DataFrame
pivoted = grouped.pivot(index=['ImageWidth', 'NumAngles', 'NumDetectors'], columns=['Type'], values='NormalisationTime')
pivoted.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in pivoted.columns.values]
pivoted.reset_index(inplace=True)

# Rename columns for clarity
pivoted = pivoted.rename(columns={
    'ImageWidth': 'Image Size',
    'NumAngles': 'Angles',
    'NumDetectors': 'Detectors',
    'Metal': 'Metal (ms)',
    'Sequential': 'Seq. (ms)',
    'OpenMP': 'OMP (ms)'
})
pivoted['Speedup: Metal vs Seq.'] = pivoted['Seq. (ms)'] / pivoted['Metal (ms)']
pivoted['Speedup: OMP vs Seq.'] = pivoted['Seq. (ms)'] / pivoted['OMP (ms)']
pivoted['Speedup: Metal vs OMP.'] = pivoted['OMP (ms)'] / pivoted['Metal (ms)']
pivoted['Sinogram Size'] = pivoted['Angles'] * pivoted['Detectors']
pivoted = pivoted[['Image Size', 'Angles', 'Detectors', 'Sinogram Size', 'Seq. (ms)','OMP (ms)', 'Metal (ms)', 'Speedup: OMP vs Seq.', 'Speedup: Metal vs Seq.', 'Speedup: Metal vs OMP.']]
styled_table = pivoted.style.format(precision=3)\
    .background_gradient(subset=['Speedup: Metal vs Seq.', 'Speedup: OMP vs Seq.', 'Speedup: Metal vs OMP.'], cmap='Blues')
styled_table
styled_table.to_html('table4.html')


    ImageWidth  NumAngles  NumDetectors      Metal     OpenMP  Sequential
0          256         90           725   5.681200   0.740333    0.789667
1          256        180           725   4.762000   1.001000    1.318333
2          256        360           725   5.532000   1.462000    2.426000
3          512         90          1449   4.747750   0.750000    2.919111
4          512        180          1449   6.564500   1.313667    2.446667
5          512        360          1449   5.113500   2.163667    4.423667
6         1024        180          2897   5.111500   2.423667    4.407333
7         1024        360          2897   5.011000   4.589667    8.730000
8         2048        360          5793   6.858500   9.284000   17.563000
9         2048        720          5793   6.844500  18.898667   35.163333
10        4096        180         11586   7.932000   8.976000   17.686500
11        4096        360         11586   7.183667  17.009667   35.496000
12        4096        720         1158

In [ ]:
from scipy.stats import zscore
# Import data
df_other = pd.read_csv('Other-Implementations/logs/performance_log.csv')
df_metal = pd.read_csv('metal-logs/metal_performance_log.csv')

# Combine both datasets into one DataFrame
df_metal['Source'] = 'Metal'
df_other['Source'] = 'Other'

# Drop rows with NAN values in ScanTime
df_other = df_other.dropna(subset=['ScanTime'])

def remove_outliers_zscore(df, column, threshold=3):
    z_scores = zscore(df[column])
    return df[abs(z_scores) < threshold]

# Remove outliers for ScanTime in Metal
df_metal = remove_outliers_zscore(df_metal, 'ScanTime')
df_other = remove_outliers_zscore(df_other, 'ScanTime')
df_combined = pd.concat([df_metal, df_other], ignore_index=True)


# Group by Type, ImageWidth,NumAngles,NumDetectors
grouped = df_combined.groupby(['ExecutionType', 'ImageWidth', 'NumAngles', 'NumDetectors'])[['ScanTime']].mean().reset_index()
# Pivot the data to create a single DataFrame
pivoted = grouped.pivot(index=['ImageWidth', 'NumAngles', 'NumDetectors'], columns=['ExecutionType'], values='ScanTime')
pivoted.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in pivoted.columns.values]
pivoted.reset_index(inplace=True)

# Rename columns for clarity
pivoted = pivoted.rename(columns={
    'ImageWidth': 'Image Size',
    'NumAngles': 'Angles',
    'NumDetectors': 'Detectors',
    'Metal': 'Metal Scan (ms)',
    'Sequential': 'Seq. Scan (ms)',
    'openmp': 'OMP Scan (ms)'
})
pivoted['Matrix Nonzero Elements'] = None 

# Add custom values for indices 0, 1, and 2
pivoted.loc[0, 'Matrix Nonzero Elements'] = 18737864  
pivoted.loc[1, 'Matrix Nonzero Elements'] = 74912415  
pivoted.loc[2, 'Matrix Nonzero Elements'] = 299563448
pivoted['Speedup: Metal vs Seq.'] = pivoted['Seq. Scan (ms)'] / pivoted['Metal Scan (ms)']
pivoted['Speedup: OMP vs Seq.'] = pivoted['Seq. Scan (ms)'] / pivoted['OMP Scan (ms)']
pivoted['Speedup: Metal vs OMP'] = pivoted['OMP Scan (ms)'] / pivoted['Metal Scan (ms)']

vmin = pivoted[['Speedup: OMP vs Seq.', 'Speedup: Metal vs Seq.', 'Speedup: Metal vs OMP']].min().min() 
vmax = pivoted[['Speedup: OMP vs Seq.', 'Speedup: Metal vs Seq.', 'Speedup: Metal vs OMP']].max().max()
pivoted = pivoted[['Image Size', 'Angles', 'Detectors', 'Matrix Nonzero Elements', 'Seq. Scan (ms)','OMP Scan (ms)', 'Metal Scan (ms)', 'Speedup: OMP vs Seq.', 'Speedup: Metal vs Seq.', 'Speedup: Metal vs OMP']]
styled_table = pivoted.style.format(precision=3)\
    .background_gradient(subset=['Speedup: Metal vs Seq.', 'Speedup: OMP vs Seq.', 'Speedup: Metal vs OMP'], cmap='Blues', vmin=vmin, vmax=vmax)
styled_table

/var/folders/3s/lc3b0s5s3l32862rhw56zrv80000gn/T/ipykernel_43789/3191878791.py:19: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_other = df_other.groupby('ExecutionType', group_keys=False).apply(


,Image Size,Angles,Detectors,Matrix Nonzero Elements,Seq. Scan (ms),OMP Scan (ms),Metal Scan (ms),Speedup: OMP vs Seq.,Speedup: Metal vs Seq.,Speedup: Metal vs OMP
0,256,90,725,18737864,58.115,33.145,29.437,1.753,1.974,1.126
1,512,90,1449,74912415,238.268,109.627,64.384,2.173,3.701,1.703
2,1024,90,2897,299563448,1467.300,763.292,412.117,1.922,3.560,1.852
